In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import os
import numpy as np
import pandas as pd
from scipy.misc import imread
from sklearn.metrics import accuracy_score
import tensorflow as tf
import imageio
import keras
from random import choice
from scipy.ndimage import rotate, shift, zoom # For data augmentation


In [3]:
root_dir = os.path.abspath('C:\\Users\\Rahul Jakhad\\Downloads\\digit recognition')


In [4]:
train = pd.read_csv(os.path.join(root_dir, 'Train', 'train.csv'))
test = pd.read_csv(os.path.join(root_dir, 'Test.csv'))

sample_submission = pd.read_csv(os.path.join(root_dir, 'Sample_Submission.csv'))

In [5]:
img_name = choice(train.filename)

In [6]:
temp = []
for img_name in train.filename:
    
    image_path = os.path.join(root_dir, 'Train', 'Images', 'train', img_name)
    img = imread(image_path, flatten=True)
    img = img.astype('float32')
    temp.append(img)
    
train_x = np.stack(temp)

E:\anaconda\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning:     `imread` is deprecated!
    `imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
    Use ``imageio.imread`` instead.
  """


In [7]:
temp = []
for img_name in test.filename:
    image_path = os.path.join(root_dir, 'Train', 'Images', 'test', img_name)
    img = imread(image_path, flatten=True)
    img = img.astype('float32')
    temp.append(img)
    
test_x = np.stack(temp)

E:\anaconda\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning:     `imread` is deprecated!
    `imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
    Use ``imageio.imread`` instead.
  after removing the cwd from sys.path.


In [8]:
train_x.shape

(49000, 28, 28)

In [9]:
test_x.shape

(21000, 28, 28)

In [10]:
Train_x = train_x.reshape(-1,28*28)

In [11]:
Test_x = train_x.reshape(-1 , 28*28)

In [12]:
train_x.shape

(49000, 28, 28)

In [13]:
test_x.shape

(21000, 28, 28)

In [14]:
y_train = train.label.values

# Data augmentation

In [15]:
def shift_in_one_direction(image, direction):
    """
    Shifts an image by one pixel in the specified direction
    """
    if direction == "DOWN":
        image = shift(image, [1, 0])
    elif direction == "UP":
        image = shift(image, [-1, 0])
    elif direction == "LEFT":
        image = shift(image, [0, -1])
    else:
        image = shift(image, [0, 1])

    return image

In [16]:
def shift_in_all_directions(image):
    """
    Shifts an image in all the directions by one pixel
    """
    reshaped_image = image.reshape(28, 28)

    down_shifted_image = shift_in_one_direction(reshaped_image, "DOWN")
    up_shifted_image = shift_in_one_direction(reshaped_image, "UP")
    left_shifted_image = shift_in_one_direction(reshaped_image, "LEFT")
    right_shifted_image = shift_in_one_direction(reshaped_image, "RIGHT")

    return (down_shifted_image, up_shifted_image,
            left_shifted_image, right_shifted_image)

In [17]:
def rotate_in_all_directions(image, angle):
    """
    Rotates an image clockwise and anti-clockwise
    """
    reshaped_image = image.reshape(28, 28)
    
    rotated_images = (rotate(reshaped_image, angle, reshape=False),
                      rotate(reshaped_image, -angle, reshape=False))
    
    return rotated_images



In [ ]:
def clipped_zoom(image, zoom_ranges):
    """
    Clips and zooms an image at the specified zooming ranges
    """
    reshaped_image = image.reshape(28, 28)
    
    h, w = reshaped_image.shape
    
    zoomed_images = []
    for zoom_range in zoom_ranges:
        zh = int(np.round(h / zoom_range))
        zw = int(np.round(w / zoom_range))
        top = (h - zh) // 2
        left = (w - zw) // 2
        
        zoomed_images.append(zoom(reshaped_image[top:top+zh, left:left+zw],
                                  zoom_range))
    
    return zoomed_images

In [25]:
def alter_image(image):
    """
    Alters an image by shifting, rotating, and zooming it
    """
    #shifted_images = shift_in_all_directions(image)
    rotated_images = rotate_in_all_directions(image, 10)
    #zoomed_images = clipped_zoom(image, [1.1, 1.2])
            
    #return np.r_[shifted_images, rotated_images , zoomed_images]
    return np.r_[rotated_images]

In [29]:
X_train_add = np.apply_along_axis(alter_image, 1, Train_x).reshape(-1, 784)
y_train_add = np.repeat(y_train,2)

In [30]:
X_train_add.shape

(98000, 784)

In [31]:
y_train_add.shape

(98000,)

In [32]:
X_train_combined = np.r_[Train_x, X_train_add]
y_train_combined = np.r_[y_train, y_train_add]


In [33]:
y_train_combined = y_train_combined.reshape((y_train_combined.shape[0] , 1))
from keras.utils import to_categorical

y_train_combined = to_categorical(y_train_combined)


In [34]:
import keras 
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers.normalization import BatchNormalization

In [35]:
X_train_combined = X_train_combined.reshape((X_train_combined.shape[0] , 28 ,28 ,1 ))

In [36]:
input_shape = (None,X_train_combined.shape[1] , X_train_combined.shape[2] , X_train_combined.shape[3] )

In [37]:
input_shape

(None, 28, 28, 1)

In [47]:
model = Sequential()

In [48]:
#Layer 1
#Conv Layer 1
model.add(Conv2D(filters = 6, 
                 kernel_size = 3, 
                 strides = 1, 
                 activation = 'relu', 
                 input_shape = (28,28,1)))

In [49]:
#Pooling layer 1
model.add(MaxPooling2D(pool_size = 2, strides = 2))

In [50]:
#Layer 2
#Conv Layer 2
model.add(Conv2D(filters = 16, 
                 kernel_size = 3,
                 strides = 1,
                 activation = 'relu',
                 input_shape = (14,14,6)))

In [51]:
#Pooling Layer 2
model.add(MaxPooling2D(pool_size = 2, strides = 2))

In [52]:
#Flatten
model.add(Flatten())

In [53]:
#Layer 3
#Fully connected layer 1
model.add(Dense(units = 120, activation = 'relu'))

In [54]:
#Layer 4
#Fully connected layer 2
model.add(Dense(units = 84, activation = 'relu'))

In [55]:
#Layer 5
#Output Layer
model.add(Dense(units = 10, activation = 'softmax'))

In [56]:
model.build(input_shape)

In [57]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 26, 26, 6)         60        
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 13, 13, 6)         0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 11, 11, 16)        880       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 5, 5, 16)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 400)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 120)               48120     
_________________________________________________________________
dense_4 (Dense)              (None, 84)               

In [62]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [63]:
y_train_combined.shape

(147000, 10)

In [64]:
X_train_combined.shape

(147000, 28, 28, 1)

In [ ]:
model.fit(X_train_combined ,y_train_combined , steps_per_epoch = 10, epochs = 42)


Epoch 1/42


In [135]:
#Converting one hot vectors to labels
labels = np.argmax(y_pred, axis = 1)

In [136]:
y_pred = model.predict(test_x)

In [137]:
labels

array([4, 0, 9, ..., 6, 6, 2], dtype=int64)

In [138]:
sample_submission.label = labels

In [139]:
sample_submission.to_csv("prediction1.csv")